# [LVV-TXXXX] - StarTracker Pointing and Tracking Test - Fixed Dome case.

Collect data with the StarTracker with the dome fixed and the telescope pointing at the South Celestial Pole.

This test is a preliminary check of collecting data before the tests with telescope movement.

**Requirements:**
 - The MTRotator should be in either ENABLED or DISABLED and the MTPtg should be ENABLED.
 - Need the `notebooks_vandv` module installed.

Please, see the [README] file for the requirements to run this notebook.  
The log messages printed in this notebook are stored in the EFD Script using the `-TTTTMMDD` format, where `TTTT` are the four last digits of the test case, `MM` is the 0-padded month, and `DD` is the 0-padded day of execution.

[README]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md
[LVV-T2730]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2730

## Setup

In [ ]:
test_case = "LVV-TXXX"
test_exec = "LVV-EXXXX"

camera_sal_indexes = [101, 102]
exposure_times = [10., 5.]  # s
base_msg = f"{test_case} {test_exec}:"

### Prepare Notebook

Start with importing libraries:

In [ ]:
%load_ext autoreload
%autoreload 2

import asyncio
import logging
import sys
import yaml

from astropy.time import Time

from lsst.ts import salobj
from lsst.ts.idl.enums.Script import ScriptState
from lsst.ts.observatory.control import RotType
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control.generic_camera import GenericCamera

from lsst.sitcom import vandv

---
Print out the execution info for future reference.

In [ ]:
exec_info = vandv.ExecutionInfo()
print(exec_info)

---

Use LOVE to put MTRotator into either ENABLED or DISABLED and MTPtg to ENABLED.

---
Setup the logger, the Domain, and the remote for the telescope control.

In [ ]:
logging.basicConfig(format="%(asctime)s %(name)s: %(message)s", level=logging.DEBUG)
log = logging.getLogger("setup")
log.level = logging.INFO

In [ ]:
os.environ["LSST_DDS_HISTORYSYNC"] = "200"
domain = salobj.Domain()
print(f"My user_host is {domain.user_host!r}\n")

Initialize the GenericCamera.

In [ ]:
camera_list = []

for index in camera_sal_indexes:

    gencam = GenericCamera(domain=domain, index=index, log=log)
    await gencam.start_task

    camera_list.append(gencam)


In [ ]:
await asyncio.gather(*[camera.start_task for camera in camera_list])

Run the cell below to hide all the messages related to the DDS and have a cleaner notebook.

In [ ]:
vandv.logger.add_filter_to_mtcs()

---
The `Script` CSC is used to record test checkpoints and allow to easy search in the EFD.

### Check CSC Status

In [ ]:
for cam in camera_list:
    await cam.assert_liveliness()
    await cam.assert_all_enabled()

## Helper Functions

In [ ]:
async def take_images(cams, exp_times, n_exps, reason):
    """Takes images with multiple cameras at the same time.
    
    cams : `list`
        Contains camera instances.
    exp_times : `list` [`float`]
        The exposure time for each camera. 
    n_exps : `list` [`float`]
        The numer of exposures for each camera.
    reason : `str`
        Reason passed to the `take_object` command.
    """ 
    reason = reason.replace(" ", "_")
    
    tasks = [asyncio.create_task(cam.take_object(exptime, n=n_exp, reason=reason))
             for (cam, exptime, n_exp) in zip(cams, exp_times, n_exps)]
    
    await asyncio.gather(*tasks)

## Data Acquisition

In [ ]:
num_exposures = [5, 5]
exposure_times = [0.5, 0.5]
reason = "Test exposures with fixed TMA"


#script.log.info(f"{base_msg} TEST of notebook for fixed TMA position with startrackers - Start")

# Take images
#script.log.info(f"{base_msg} Take {num_exposures} image(s) with both star trackers - Start")
await take_images(camera_list, exposure_times, num_exposures, reason)
#script.log.info(f"{base_msg} Take {num_exposures} image(s) with both star trackers - Done")


#script.log.info(f"{base_msg} TEST of notebook for fixed TMA position with startrackers - Stop")